# DeepDream



## Importing libraries

In [ ]:
!pip3 install tensorflow-gpu==1.15.2

     |████████████████████████████████| 411.0MB 38kB/s 
     |████████████████████████████████| 3.8MB 47.0MB/s 
     |████████████████████████████████| 512kB 47.0MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=7e40dcc1e090fa401d4ddba5a8302f75f2cf796ffffc5af7278626becab80889
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow 2.2.0 has requirement gast==0.3.3, but you'll have gast 0.2.2 which is incompatible.
ERROR: tensorflow 2.2.0 has requirement tensorboard<2.3.0,>=2.2.0, but you'll have tensorboard 1.15.0 which is incompatible.
ERROR: tensorflow 2.2.0 has requirement tensorflow-estimator<2.3.0,>=2.2.0, but you'll have tensorflow-estimator 1.15.1 which is incompatible.
ERROR: tensorflow-probability 0.10.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.

In [ ]:
!mkdir input_images
!mv barcelona.jpg input_images/barcelona.jpg

In [ ]:
!mv messi_2020.jpeg input_images/
!mv ney.jpg input_images/

In [ ]:
!mkdir final_images

In [ ]:
from tensorflow.keras.applications import inception_v3
import tensorflow.keras.backend as K
import numpy as np
import matplotlib.pyplot as plt
import scipy
from tensorflow.keras.preprocessing import image

## Loading model

In [ ]:
K.clear_session()
K.set_learning_phase(0)
model = inception_v3.InceptionV3(weights='imagenet', include_top=False)
model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, None, None, 3 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, None, None, 3 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, None, None, 3 0           batch_normalization[0][0]        
______

In [ ]:
layer_contributions = {'mixed1' : 0.2, 'mixed2' : 2.0, 'mixed3' : 3.0, 'mixed6' : 0.2, 'mixed8' : 1.5, 'mixed10' : 2.1} # Tweak according to liking

## Defining loss

In [ ]:
layer_dict = dict([(layer.name, layer) for layer in model.layers])
loss = K.variable(0.0)
for  layer_name in layer_contributions:
    coeff = layer_contributions[layer_name]
    activation = layer_dict[layer_name].output

    scaling = K.prod(K.cast(K.shape(activation), 'float32'))
    loss = loss + coeff * K.sum(K.square(activation[:, 2: -2, 2: -2, :])) / scaling

## Gradient Ascent

In [ ]:
dream = model.input
grads = K.gradients(loss, dream)[0]
grads /= K.maximum(K.mean(K.abs(grads)), 1e-7) # Normalisation trick (Dividing by maximum MAE)
outputs = [loss, grads]
fetch_loss_and_grads = K.function([dream], outputs)

def eval_loss_and_grads(x):
    outs = fetch_loss_and_grads([x])
    loss_value = outs[0]
    grad_values = outs[1]
    return loss_value, grad_values

def gradient_ascent(x, iterations, step, max_loss=None):
    for i in range(iterations):
        loss_value, grad_values = eval_loss_and_grads(x)
        if max_loss is not None and loss_value > max_loss:
            break
        print('\t Loss value at ', i, ': ', loss_value)
        x += step * grad_values
    return x

## Defining auxillary functions

In [ ]:
import imageio
def preprocess_image(image_path):
  img = image.load_img(image_path)
  img = image.img_to_array(img)
  img = np.expand_dims(img, axis=0)
  img = inception_v3.preprocess_input(img)
  return img

def deprocess_image(x):
  if K.image_data_format() == 'channels_first':
    x = x.reshape((3, x.shape[2], x.shape[3]))
    x = x.transpose((1, 2, 0))
  else:
    x = x.reshape((x.shape[1], x.shape[2], 3))
  x /= 2.0
  x += 0.5
  x *= 255
  x = np.clip(x, 0, 255).astype('uint8')
  return x

def resize_img(img, size):
  img = np.copy(img)
  factors = (1, float(size[0])/img.shape[1], float(size[1])/img.shape[2], 1)
  return scipy.ndimage.zoom(img, factors, order=1)

def save_img(img, fname):
  pil_img = deprocess_image(np.copy(img))
  imageio.imsave(fname, pil_img)

## Running grad ascent over different octaves

In [ ]:
step = 0.01
num_octave = 4
octave_scale = 1.4
iterations = 20
max_loss = 10.0
base_image_path = 'input_images/barcelona.jpg'
img = preprocess_image(base_image_path)

original_shape = img.shape[1:3]
successive_shapes = [original_shape]
for i in range(1, num_octave):
    shape = tuple([int(dim / (octave_scale ** i)) for dim in original_shape])
    successive_shapes.append(shape)

successive_shapes = successive_shapes[::-1]

original_img = np.copy(img)
shrunk_original_img = resize_img(img, successive_shapes[0])

for shape in successive_shapes:
    print('Processing image shape: ', shape)
    img = resize_img(img, shape)
    img = gradient_ascent(img, iterations=iterations, step=step, max_loss=max_loss)

    upscaled_shrunk_original_img = resize_img(shrunk_original_img, shape)
    same_size_original = resize_img(original_img, shape)
    lost_detail = same_size_original - upscaled_shrunk_original_img

    img = img + lost_detail
    shrunk_original_img = resize_img(original_img, shape)

save_img(img, fname='final_images/barcelona.png')

Processing image shape:  (265, 364)
	 Loss value at  0 :  2.3890498
	 Loss value at  1 :  3.265152
	 Loss value at  2 :  3.7781553
	 Loss value at  3 :  3.6367092
	 Loss value at  4 :  3.9696693
	 Loss value at  5 :  4.574601
	 Loss value at  6 :  4.9728756
	 Loss value at  7 :  4.623268
	 Loss value at  8 :  6.489656
	 Loss value at  9 :  6.7490854
	 Loss value at  10 :  6.1771927
	 Loss value at  11 :  6.508789
	 Loss value at  12 :  7.183971
Processing image shape:  (371, 510)
	 Loss value at  0 :  2.5407667
	 Loss value at  1 :  3.2409825
	 Loss value at  2 :  4.033254
	 Loss value at  3 :  4.310517
	 Loss value at  4 :  4.6385465
	 Loss value at  5 :  4.9894996
	 Loss value at  6 :  5.379633
	 Loss value at  7 :  5.8846283
	 Loss value at  8 :  6.05024
	 Loss value at  9 :  6.385705
	 Loss value at  10 :  6.6083946
	 Loss value at  11 :  6.7601013
	 Loss value at  12 :  7.3157015
	 Loss value at  13 :  7.4808707
	 Loss value at  14 :  8.418463
	 Loss value at  15 :  8.375801
	 Los

In [ ]:
!zip -r /content/file.zip /content/final_images

  adding: content/final_images/ (stored 0%)
  adding: content/final_images/ney_final.png (deflated 0%)
  adding: content/final_images/barcelona.png (deflated 0%)
  adding: content/final_images/messi_2020_final_dream.png (deflated 0%)
  adding: content/final_images/.ipynb_checkpoints/ (stored 0%)


In [ ]:
from google.colab import files
files.download("/content/file.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>